In [1]:
import os
import sys

import github as gh

from selenium import webdriver

from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

import time
from tqdm import tqdm
import pickle
from uuid import uuid4

import re

def generate_test_data():
    mock_repository_url='https://github.com/bamblebam/image-classification-rps'
    mock_python_file_url=f'{mock_repository_url}/blob/master/testmodels.py'
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(
        ChromeDriverManager(path="./").install(), options=options
    )  # downloads the latest version of the chrome drivers
    driver.get(mock_python_file_url)
    time.sleep(2.5)
    code_body = driver.find_element_by_xpath(
                    "//*[@class='highlight tab-size js-file-line-container']"  # xpath for code container
                )
    with open('test_code.txt','w') as f:
        f.write(code_body.text)
    
generate_test_data()



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Get LATEST driver version for 92.0.4515
Trying to download new driver from https://chromedriver.storage.googleapis.com/92.0.4515.43/chromedriver_win32.zip
Driver has been saved in cache [./\drivers\chromedriver\win32\92.0.4515.43]


In [2]:
def getLayerSequence1helper(code):
    """
    Helper function to get models of type Sequential([...])
    :param code: the code from the .py file
    :return: list in correct sequence of layers
    :rtype: list model.add(.*)
    """
    rawLayerSequence = re.findall("Sequential\(\[([^]]+)\]\)", code, re.DOTALL)
    layerSequence = list()
    temp = list()
    for i in rawLayerSequence:
        temp = re.findall(".?(.*)\)", i)
        layerSequence.append(temp)
    return layerSequence

In [3]:
with open('test_code.txt') as f:
    code = f.read()
    f.close()
output=getLayerSequence1helper(code)
print(output)

[[" keras.layers.Conv2D(128, (3, 3), input_shape=(64, 64, 1), padding='same'", " keras.layers.Conv2D(128, (3, 3), input_shape=(64, 64, 1), padding='same'", " keras.layers.Conv2D(128, (3, 3), input_shape=(64, 64, 1), padding='same'", " keras.layers.Conv2D(128, (3, 3), input_shape=(64, 64, 1), padding='same'", " keras.layers.Dense(3, activation='softmax'", " keras.layers.Dense(3, activation='softmax'", " keras.layers.Dense(3, activation='softmax'"], [" Conv2D(128, (3, 3), input_shape=(64, 64, 1), padding='same'", " Conv2D(128, (3, 3), input_shape=(64, 64, 1), padding='same'", " Conv2D(128, (3, 3), input_shape=(64, 64, 1), padding='same'", " Conv2D(128, (3, 3), input_shape=(64, 64, 1), padding='same'", " Dense(3, activation='softmax'", " Dense(3, activation='softmax'", " Dense(3, activation='softmax'"]]
